In [1]:
import ast
import os
import shutil

import cv2
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Image, clear_output
from sklearn.model_selection import StratifiedKFold, train_test_split
from tqdm import tqdm
from PIL import Image
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
import pydicom as dicom

In [9]:
# !git clone https://github.com/WongKinYiu/ScaledYOLOv4.git
# !git clone https://github.com/thomasbrandon/mish-cuda
%cd /app/_data/ScaledYOLOv4/mish-cuda
!python setup.py build install
clear_output()
%cd /app/_data/ScaledYOLOv4
! git checkout yolov4-large

/app/_data/ScaledYOLOv4
M	detect.py
M	utils/datasets.py
M	utils/general.py
Already on 'yolov4-large'
Your branch is up to date with 'origin/yolov4-large'.


In [10]:
labels = {"negative": 0, "typical": 1, "indeterminate": 2, "atypical": 3, "opacity": 4}

# preparing images

In [11]:
dicom_df = pd.read_csv("/app/_data/train.csv")

In [12]:
dicom_df['image'] = dicom_df['image'].str[:-4]

In [13]:
list_wrong = dicom_df[(dicom_df["class"] != "negative") & (dicom_df["label"] == "none 1 0 0 1 1")][
    "id_image"
].tolist()
dicom_df = dicom_df.query("id_image not in @list_wrong").reset_index(drop=True)

In [18]:
resized_yolo_path = "/app/_data/yolo4_dataset/resized_yolo_1024_npy/images/"
os.makedirs(resized_yolo_path)

In [19]:
# for ix in tqdm(dicom_df.index.tolist()):
#     img_path = dicom_df.loc[ix, 'path']
#     data_file = dicom.dcmread(img_path)
#     image_name = dicom_df.loc[ix, 'image']
#     img = data_file.pixel_array
#     if dicom_df.loc[ix, 'PhotometricInterpretation'] == "MONOCHROME1":
#         img = img.max() - img
#     img = (img - img.min()) / (img.max() - img.min())
#     img = (np.array(img) * 255).astype("uint8")
#     img = np.stack([img, img, img], axis=-1)
#     h0, w0 = img.shape[:2]
#     r = 1024 / max(h0, w0)
#     img_yolo = cv2.resize(img, (int(w0 * r), int(h0 * r)), interpolation=cv2.INTER_LINEAR)
#     img_yolo = Image.fromarray(img_yolo)
#     img_yolo.save(resized_yolo_path + image_name+'.png', format = 'PNG')

## preparing data

In [20]:
df = pd.read_csv("/app/_data/bboxes.csv")
df["path"] = resized_yolo_path + df["img"] + ".png"
base_config = {
    "IMG_SIZE": 1024,
    "BATCH_SIZE": 10,
    "SEED": 42,
}

In [21]:
df = df.query("img not in @list_wrong").reset_index(drop=True)
df

,study,img,label,bbox,x_center,y_center,width,height,class,path
0,5776db0cec75,000a312787f2,typical,1,0.306066,0.441824,0.241226,0.549685,0,/app/_data/yolo4_dataset/resized_yolo_1024_npy...
1,5776db0cec75,000a312787f2,typical,1,0.656307,0.422013,0.257204,0.505031,0,/app/_data/yolo4_dataset/resized_yolo_1024_npy...
2,ff0879eb20ed,000c3a3f293f,negative,0,NaN,NaN,NaN,NaN,0,/app/_data/yolo4_dataset/resized_yolo_1024_npy...
3,9d514ce429a7,0012ff7358bc,typical,1,0.363652,0.274319,0.283965,0.392996,0,/app/_data/yolo4_dataset/resized_yolo_1024_npy...
4,9d514ce429a7,0012ff7358bc,typical,1,0.687567,0.394942,0.201907,0.473411,0,/app/_data/yolo4_dataset/resized_yolo_1024_npy...
...,...,...,...,...,...,...,...,...,...,...
9584,7eed9af03814,ffd9b6cf2961,typical,1,0.296171,0.480181,0.128571,0.355606,0,/app/_data/yolo4_dataset/resized_yolo_1024_npy...
9585,a0cb0b96fb3d,ffdc682f7680,typical,1,0.817057,0.468622,0.351563,0.746727,0,/app/_data/yolo4_dataset/resized_yolo_1024_npy...
9586,a0cb0b96fb3d,ffdc682f7680,typical,1,0.314128,0.565537,0.155599,0.222429,0,/app/_data/yolo4_dataset/resized_yolo_1024_npy...
9587,7d82d53204b8,ffe942c8655f,typical,1,0.313157,0.355756,0.325139,0.550925,0,/app/_data/yolo4_dataset/resized_yolo_1024_npy...


In [22]:
groupped_data = pd.pivot_table(
    df, index="img", values=["label", "bbox", "path"], aggfunc="first"
)
groupped_data["img"] = groupped_data.index
groupped_data = groupped_data.reset_index(drop=True)
groupped_data.head()
groupped_data["bbox"].value_counts()

,bbox,label,path,img
0,1,typical,/app/_data/yolo4_dataset/resized_yolo_1024_npy...,000a312787f2
1,0,negative,/app/_data/yolo4_dataset/resized_yolo_1024_npy...,000c3a3f293f
2,1,typical,/app/_data/yolo4_dataset/resized_yolo_1024_npy...,0012ff7358bc
3,1,atypical,/app/_data/yolo4_dataset/resized_yolo_1024_npy...,001398f4ff4f
4,1,typical,/app/_data/yolo4_dataset/resized_yolo_1024_npy...,001bd15d1891


1    4294
0    1736
Name: bbox, dtype: int64

In [23]:
skf = StratifiedKFold(n_splits=5, random_state=base_config["SEED"], shuffle=True)
train_ids = []
val_ids = []
for train_index, valid_index in skf.split(groupped_data, groupped_data["label"]):
    train_ids.append(train_index)
    val_ids.append(valid_index)

In [24]:
if not os.path.exists("/app/_data/yolo4_dataset/"):
    os.mkdir("/app/_data/yolo4_dataset/")
for i in range(5):
    train_img_path = groupped_data.loc[train_ids[i]]["path"].tolist()
    val_img_path = groupped_data.loc[val_ids[i]]["path"].tolist()
    np.savetxt(
        "/app/_data/yolo4_dataset/train_" + str(i) + ".txt", train_img_path, fmt="%s"
    )
    np.savetxt(
        "/app/_data/yolo4_dataset/val_" + str(i) + ".txt", val_img_path, fmt="%s"
    )

In [25]:
for img_name in tqdm(df["img"].unique().tolist()):
    new_df = df[df["img"] == img_name].reset_index(drop=True)
    list_boxes = []
    if new_df.loc[0, "bbox"] == 1:
        boxes = new_df[["x_center", "y_center", "width", "height"]].values
        for box in boxes:
            list_boxes.append(
                [
                    str(labels["opacity"]),
                    str(box[0]),
                    str(box[1]),
                    str(box[2]),
                    str(box[3]),
                ]
            )
    list_boxes.append([str(labels[new_df.loc[0, "label"]]), "0.5", "0.5", "1", "1"])
    np.savetxt(
        "/app/_data/yolo4_dataset/resized_yolo_1024_npy/labels/" + img_name + ".txt",
        list_boxes,
        fmt="%s",
    )
clear_output()

## Writing data.yaml and custom parameters yolov4-p7

In [27]:
# customize iPython writefile so we can write variables

from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [28]:
if not os.path.exists("/app/_data/ScaledYOLOv4/data/DataFile"):
    os.mkdir("/app/_data/ScaledYOLOv4/data/DataFile")

In [29]:
%%writetemplate /app/_data/ScaledYOLOv4/data/DataFile/data.yaml
# train and val datasets (image directory or *.txt file with image paths)
train: /app/_data/yolo4_dataset/train_0.txt
val: /app/_data/yolo4_dataset/val_0.txt
# number of classes
nc: 5
# class names
names: ["negative", "typical", "indeterminate", "atypical", "opacity"]

In [30]:
%%writetemplate /app/_data/ScaledYOLOv4/data/DataFile/customYOLOv4p7.yaml
# parameters yolov4-p7
nc: 5  # number of classes
depth_multiple: 1.0  # expand model depth
width_multiple: 1.25  # expand layer channels

# anchors
anchors:
  - [13,17,  22,25,  27,66,  55,41]  # P3/8
  - [57,88,  112,69,  69,177,  136,138]  # P4/16
  - [136,138,  287,114,  134,275,  268,248]  # P5/32
  - [268,248,  232,504,  445,416,  640,640]  # P6/64
  - [812,393,  477,808,  1070,908,  1408,1408]  # P7/128

# csp-p7 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
   [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2
   [-1, 1, BottleneckCSP, [64]],
   [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 5-P3/8
   [-1, 15, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 7-P4/16
   [-1, 15, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]], # 9-P5/32
   [-1, 7, BottleneckCSP, [1024]],
   [-1, 1, Conv, [1024, 3, 2]], # 11-P6/64
   [-1, 7, BottleneckCSP, [1024]],
   [-1, 1, Conv, [1024, 3, 2]], # 13-P7/128
   [-1, 7, BottleneckCSP, [1024]],  # 14
  ]

# yolov4-p7 head
# na = len(anchors[0])
head:
  [[-1, 1, SPPCSP, [512]], # 15
   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-6, 1, Conv, [512, 1, 1]], # route backbone P6
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [512]], # 20 
   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-13, 1, Conv, [512, 1, 1]], # route backbone P5
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [512]], # 25
   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-20, 1, Conv, [256, 1, 1]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [256]], # 30
   [-1, 1, Conv, [128, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [-27, 1, Conv, [128, 1, 1]], # route backbone P3
   [[-1, -2], 1, Concat, [1]],
   [-1, 3, BottleneckCSP2, [128]], # 35
   [-1, 1, Conv, [256, 3, 1]],
   [-2, 1, Conv, [256, 3, 2]],
   [[-1, 30], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [256]], # 39
   [-1, 1, Conv, [512, 3, 1]],
   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 25], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [512]], # 43
   [-1, 1, Conv, [1024, 3, 1]],
   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 20], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [512]], # 47
   [-1, 1, Conv, [1024, 3, 1]],
   [-2, 1, Conv, [512, 3, 2]],
   [[-1, 15], 1, Concat, [1]],  # cat
   [-1, 3, BottleneckCSP2, [512]], # 51
   [-1, 1, Conv, [1024, 3, 1]],

   [[36,40,44,48,52], 1, Detect, [nc, anchors]],   # Detect(P3, P4, P5, P6, P7)
  ]

## train

In [32]:
!python train.py --img 1024 \
                 --batch 6 \
                 --epochs 150 \
                 --data /app/_data/ScaledYOLOv4/data/DataFile/data.yaml \
                 --cfg /app/_data/ScaledYOLOv4/data/DataFile/customYOLOv4p7.yaml \
                 --weights /app/_data/ScaledYOLOv4/weights/yolov4-p7.pt \
                 --name 5cl_yolov4p7_42 \
                 --cache

Using CUDA device0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)

Namespace(adam=False, batch_size=6, bucket='', cache_images=True, cfg='/app/_data/ScaledYOLOv4/data/DataFile/customYOLOv4p7.yaml', data='/app/_data/ScaledYOLOv4/data/DataFile/data.yaml', device='', epochs=150, evolve=False, global_rank=-1, hyp='data/hyp.finetune.yaml', img_size=[1024, 1024], local_rank=-1, logdir='runs/', multi_scale=False, name='5cl_yolov4p7_42', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=6, weights='/app/_data/ScaledYOLOv4/weights/yolov4-p7.pt', world_size=1)
Start Tensorboard with "tensorboard --logdir runs/", view at http://localhost:6006/
Hyperparameters {'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'transl